# 데이콘 운동동작분류 - 2등 분석 (21.03.23)

###  1D-CNN에 global-average-pooling 사용
### 타겟 변수가 굉장히 불균형하기 때문에 Stratified KFold, K=10

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings
warnings.filterwarnings(action='ignore')

load data

In [23]:
train=pd.read_csv('train_features.csv')
train_labels=pd.read_csv('train_labels.csv')
test=pd.read_csv('test_features.csv')
submission=pd.read_csv('sample_submission.csv')
pd.options.display.max_columns=50

In [2]:
train=pd.read_csv('./data/train_features.csv')
train_labels=pd.read_csv('./data/train_labels.csv')
test=pd.read_csv('./data/test_features.csv')

submission=pd.read_csv('./data/sample_submission.csv')

pd.options.display.max_columns=50

FileNotFoundError: [Errno 2] File ./data/train_features.csv does not exist: './data/train_features.csv'

Feature engneering
가속도, 자이로, (자이로-가속도) 센서값을 에너지로 표현

In [24]:
# 가속도
train['acc_Energy']=(train['acc_x']**2+train['acc_y']**2+train['acc_z']**2)**(1/3)
test['acc_Energy']=(test['acc_x']**2+test['acc_y']**2+test['acc_z']**2)**(1/3)

# 자이로
train['gy_Energy']=(train['gy_x']**2+train['gy_y']**2+train['gy_z']**2)**(1/3)
test['gy_Energy']=(test['gy_x']**2+test['gy_y']**2+test['gy_z']**2)**(1/3)

# (자이로-가속도)
train['gy_acc_Energy']=((train['gy_x']-train['acc_x'])**2+(train['gy_y']-train['acc_y'])**2+(train['gy_z']-train['acc_z'])**2)**(1/3)
test['gy_acc_Energy']=((test['gy_x']-test['acc_x'])**2+(test['gy_y']-test['acc_y'])**2+(test['gy_z']-test['acc_z'])**2)**(1/3)

id별 데이터는 0.02초마다 측정된 값들이기 때문에 이전 시간 대비 변화량 적용

In [25]:
dt=0.02 # 0.02초
def jerk_signal(signal): # 이전 시간 대비 변화량 = (현재값-이전값)/0.02 , (시그널의 길이-1)만큼 반복 ex) 5개 -> 4번 반복
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [26]:
train_dt=[]
# id에서 중복값 제거하고(id는 3125개) 진행표시바(중간중간 잘 생성이 되었나 확인 목적) 생성
for i in tqdm(train['id'].unique()): # for i in train['id'].unique() -> 3125
    # train['id']=i인 것의 행 반환 ex) id=1인 것의 acc_x : 000, acc_y : 000, ..., gy_z : 000
    temp=train.loc[train['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        # np.insert():주어진 축에 대해 주어진 인덱스 앞에 값을 삽입
        # values의 첫번째 위치(index=0)에 값 0 삽입
        values=np.insert(values,0,0)
        # temp에서 v열(2열~8열) (_dt붙이고) : (acc_x_dt, ..., gy_z_dt)
        temp.loc[:,v+'_dt']=values
    train_dt.append(temp)

100%|██████████████████████████████████████████████████████████████████████████████| 3125/3125 [01:55<00:00, 27.03it/s]


In [18]:
a = jerk_signal(temp[v].values)
print(a)

[ 2.82558698e+01  3.07393263e+01  3.77036863e+01  3.27842580e+01
  2.74148381e+01  3.53146369e+01  2.03802473e+01  1.13137897e+01
  3.56101443e+01  3.28753839e+01  1.59521282e+01  6.85638255e+00
  4.00210319e+00 -2.89306828e+00 -1.84113796e+01 -3.52545613e+01
 -4.80210679e+01 -3.26848757e+01 -2.08209879e+01 -4.51372108e+01
 -3.96327407e+01 -8.16996619e+00 -2.76432860e+01 -4.12402212e+01
 -4.15427923e+01 -3.48196037e+01 -1.90412726e+01 -2.58327342e+01
 -2.01566753e+01 -1.78700078e+01 -4.35200631e+01 -6.77132161e+01
 -8.56964652e+01 -1.03578164e+02 -7.35790136e+01 -7.65701750e+01
 -1.12670938e+02 -9.90975003e+01 -7.56839937e+01  1.07458571e+01
  3.46051058e+01  3.15185229e+01  7.09888272e+01  8.96541771e+01
  9.10091641e+01  5.48066526e+01  2.34990586e+01  7.47722017e+00
  9.64479815e-01  2.41969844e+01 -3.11481490e+00 -1.53887224e+01
 -1.18177560e+01 -4.97182296e+00 -3.26608505e+00  3.67337531e+01
  5.12474782e+01  2.91456857e+01  6.39685337e+01  8.70152799e+01
  5.33013350e+01  5.36923

In [19]:
b = np.insert(a,0,0)
print(b)

[ 0.00000000e+00  2.82558698e+01  3.07393263e+01  3.77036863e+01
  3.27842580e+01  2.74148381e+01  3.53146369e+01  2.03802473e+01
  1.13137897e+01  3.56101443e+01  3.28753839e+01  1.59521282e+01
  6.85638255e+00  4.00210319e+00 -2.89306828e+00 -1.84113796e+01
 -3.52545613e+01 -4.80210679e+01 -3.26848757e+01 -2.08209879e+01
 -4.51372108e+01 -3.96327407e+01 -8.16996619e+00 -2.76432860e+01
 -4.12402212e+01 -4.15427923e+01 -3.48196037e+01 -1.90412726e+01
 -2.58327342e+01 -2.01566753e+01 -1.78700078e+01 -4.35200631e+01
 -6.77132161e+01 -8.56964652e+01 -1.03578164e+02 -7.35790136e+01
 -7.65701750e+01 -1.12670938e+02 -9.90975003e+01 -7.56839937e+01
  1.07458571e+01  3.46051058e+01  3.15185229e+01  7.09888272e+01
  8.96541771e+01  9.10091641e+01  5.48066526e+01  2.34990586e+01
  7.47722017e+00  9.64479815e-01  2.41969844e+01 -3.11481490e+00
 -1.53887224e+01 -1.18177560e+01 -4.97182296e+00 -3.26608505e+00
  3.67337531e+01  5.12474782e+01  2.91456857e+01  6.39685337e+01
  8.70152799e+01  5.33013

In [14]:
temp

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_Energy,gy_Energy,gy_acc_Energy,acc_x_dt,acc_y_dt,acc_z_dt,gy_x_dt,gy_y_dt,gy_z_dt,acc_Energy_dt,gy_Energy_dt,gy_acc_Energy_dt
1874400,3124,0,-0.862090,-0.448951,0.325390,17.735992,-87.798082,-64.138319,1.016601,22.980950,22.977501,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1874401,3124,1,-0.840665,-0.491719,0.362417,11.027550,-92.245611,-66.637434,1.025938,23.556320,23.542618,1.071271,-2.138419,1.851341,-335.422139,-222.376462,-124.955789,0.466858,28.768515,28.255870
1874402,3124,2,-0.817442,-0.528205,0.385158,7.754232,-96.420087,-69.106258,1.030889,24.177296,24.157405,1.161117,-1.824307,1.137080,-163.665861,-208.723805,-123.441191,0.247549,31.048789,30.739326
1874403,3124,3,-0.792708,-0.558292,0.390222,5.103146,-101.871034,-71.446932,1.029881,24.937665,24.911479,1.236734,-1.504336,0.253176,-132.554304,-272.547359,-117.033705,-0.050411,38.018473,37.703686
1874404,3124,4,-0.759260,-0.594246,0.404468,-0.829132,-107.364209,-72.485246,1.030148,25.602368,25.567164,1.672364,-1.797704,0.712302,-296.613915,-274.658737,-51.915694,0.013361,33.235119,32.784258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437,1.009050,25.963234,25.897316,1.484646,0.303666,0.800069,-150.644663,-34.630282,-8.380088,-0.679712,8.387109,8.432977
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087,1.002827,25.784692,25.722482,1.474659,-0.005442,1.775771,-39.061611,110.842743,-16.732496,-0.311171,-8.927089,-8.741727
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125,1.006239,25.628060,25.572145,0.915321,-0.407957,1.744566,113.799702,151.036858,-137.001896,0.170620,-7.831611,-7.516832
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478,1.001038,25.626266,25.573288,1.709833,-0.796984,0.479107,211.827245,-18.171144,-44.717652,-0.260074,-0.089713,0.057150


In [13]:
train_dt

[     id  time     acc_x     acc_y     acc_z       gy_x       gy_y       gy_z  \
 0     0     0  1.206087 -0.179371 -0.148447  -0.591608 -30.549010 -31.676112   
 1     0     1  1.287696 -0.198974 -0.182444   0.303100 -39.139103 -24.927216   
 2     0     2  1.304609 -0.195114 -0.253382  -3.617278 -44.122565 -25.019629   
 3     0     3  1.293095 -0.230366 -0.215210   2.712986 -53.597843 -27.454013   
 4     0     4  1.300887 -0.187757 -0.222523   4.286707 -57.906561 -27.961234   
 ..   ..   ...       ...       ...       ...        ...        ...        ...   
 595   0   595  0.985242 -0.326122 -0.354528 -14.903280  20.172339  22.973018   
 596   0   596  1.052837 -0.220710 -0.413472 -10.857025  19.786856  23.174597   
 597   0   597  1.025643 -0.227845 -0.354516  -2.334243  25.768654  18.932070   
 598   0   598  1.031553 -0.387862 -0.277857  -9.710746  28.697694  20.631577   
 599   0   599  1.138159 -0.426846 -0.430263 -15.891015  21.675950  32.123007   
 
      acc_Energy  gy_Energ

In [27]:
test_dt=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    test_dt.append(temp)

100%|████████████████████████████████████████████████████████████████████████████████| 782/782 [00:23<00:00, 33.70it/s]


가속도, 자이로 센서값들을 푸리에 변환
푸리에 변환(Fourier transform, FT)은 시간이나 공간에 대한 함수를 시간 또는 공간 주파수 성분으로 분해하는 변환을 말한다. 

In [28]:
from scipy import fftpack
from numpy.fft import *

def fourier_transform_one_signal(t_signal):
    # scipy.fftpack.fft ()를 사용하여 FFT(fast Fourier transforms)를 계산
    complex_f_signal= fftpack.fft(t_signal)
    # fft() 명령어를 수행한 결과를 살펴보면 파형을 실수부와 허수부로 나뉘는 역할만을 하게 됩니다.
    # 주파수 해석의 목적은 각 주파수 별로 크기 (magnitude)가 어느 정도인지 알아내는 것이기 때문에
    # 절대값 함수 abs()를 이용해서 크기를 연산할 수 있습니다.
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal


In [29]:
train=pd.concat(train_dt)

In [30]:
train.head()

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_Energy,gy_Energy,gy_acc_Energy,acc_x_dt,acc_y_dt,acc_z_dt,gy_x_dt,gy_y_dt,gy_z_dt,acc_Energy_dt,gy_Energy_dt,gy_acc_Energy_dt
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,1.146962,12.465436,12.427938,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,1.200703,12.913284,12.865692,4.080495,-0.980114,-1.699854,44.735403,-429.504677,337.444793,2.687024,22.392358,21.887693
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,1.217403,13.725729,13.692643,0.845632,0.192961,-3.546937,-196.018888,-249.173073,-4.620631,0.835012,40.622253,41.347563
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,1.209981,15.374021,15.314907,-0.575711,-1.762585,1.908626,316.513181,-473.763910,-121.719195,-0.371100,82.414636,81.113199
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,1.211254,16.074363,16.017964,0.389598,2.130453,-0.365665,78.686055,-215.435892,-25.361098,0.063656,35.017060,35.152822


In [31]:
fft=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for i in train.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft.append(temp)
train=pd.concat(fft)

100%|█████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:25<00:00, 123.46it/s]


In [32]:
test=pd.concat(test_dt)

In [33]:
fft_t=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for i in test.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft_t.append(temp)
test=pd.concat(fft_t)

100%|███████████████████████████████████████████████████████████████████████████████| 782/782 [00:03<00:00, 199.63it/s]


Standard scaling 적용

In [34]:
col=train.columns
train_s=train.copy()
test_s=test.copy()

In [35]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
# 평균 0 , 분산 1로 조정
scaler = StandardScaler()

train_s.iloc[:,2:]= scaler.fit_transform(train_s.iloc[:,2:])
train_sc = pd.DataFrame(data = train_s,columns =col)

test_s.iloc[:,2:]= scaler.transform(test_s.iloc[:,2:])
test_sc = pd.DataFrame(data = test_s,columns =col)
# train_sc, test_sc

In [36]:
train_sc.head()

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_Energy,gy_Energy,gy_acc_Energy,acc_x_dt,acc_y_dt,acc_z_dt,gy_x_dt,gy_y_dt,gy_z_dt,acc_Energy_dt,gy_Energy_dt,gy_acc_Energy_dt
0,0,0,27.356382,8.807207,19.465910,0.376992,0.869226,0.150423,0.495681,-0.272719,-0.276391,0.000027,0.000298,-0.000433,0.000347,0.000373,0.000273,0.000101,0.001505,0.001501
1,0,1,-0.054866,0.833464,0.820412,-0.282128,-0.093560,0.011266,0.742974,-0.236152,-0.240632,0.416836,-0.118821,-0.255054,0.032738,-0.349095,0.377085,0.564992,0.166566,0.162871
2,0,2,0.024046,0.315921,0.081086,-0.182551,-0.053585,-0.003708,0.819822,-0.169815,-0.173080,0.086405,0.023750,-0.531727,-0.141582,-0.202368,-0.004887,0.175645,0.300944,0.306341
3,0,3,0.065632,0.117634,-0.040874,-0.194863,0.154242,0.005408,0.785669,-0.035229,-0.040560,-0.058780,-0.213920,0.285459,0.229520,-0.385106,-0.135647,-0.077915,0.609008,0.599518
4,0,4,0.151477,0.300751,0.317742,-0.350724,0.494539,0.154354,0.791528,0.021954,0.016872,0.039823,0.259227,-0.055206,0.057320,-0.174917,-0.028047,0.013483,0.259626,0.260669


모델링
CNN, LSTM, CNN+LSTM 등 여러 구조 적용해보다가 CNN에서 Flatten 없이 Global average pooling 한 구조가 가장 성능이 좋아 채택했습니다.

In [38]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_addons as tfa
from keras.models import Sequential
from keras.layers import Dense, LSTM,Bidirectional,Dropout
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from keras import backend as K 
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from sklearn.model_selection import KFold,StratifiedKFold
from numpy.random import seed
import keras

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\leesa\anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\leesa\anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\leesa\anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\leesa\anaconda3\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\leesa\anaconda3\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: DLL 초기화 루틴을 실행할 수 없습니다.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\leesa\anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\leesa\anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\leesa\anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\leesa\anaconda3\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\leesa\anaconda3\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: DLL 초기화 루틴을 실행할 수 없습니다.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C

TypeError: can only concatenate str (not "list") to str

In [39]:
X=np.array(train_sc.iloc[:,2:]).reshape(3125, 600, -1)
X.shape

(3125, 600, 18)

In [40]:
test_x=np.array(test_sc.iloc[:,2:]).reshape(782, 600, -1)
test_x.shape

(782, 600, 18)

In [41]:
y = train_labels['label'].values
# tf.keras.utils.to_categorical : Converts a class vector (integers) to binary class matrix.
y = tf.keras.utils.to_categorical(train_labels['label']) 
y.shape

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\leesa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-41-15396b720017>", line 3, in <module>
    y = tf.keras.utils.to_categorical(train_labels['label'])
NameError: name 'tf' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\leesa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\leesa\anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\leesa\anaco

NameError: name 'tf' is not defined

In [42]:
# cnn model -> pytorch 구현
def cnn_model(input_shape, classes):
    seed(2021)
    tf.random.set_seed(2021)
    
    input_layer = keras.layers.Input(input_shape)
    conv1 = keras.layers.Conv1D(filters=128, kernel_size=9, padding='same')(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activation='relu')(conv1)
    conv1 = keras.layers.Dropout(rate=0.3)(conv1)

    conv2 = keras.layers.Conv1D(filters=256, kernel_size=6, padding='same')(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)
    conv2 = keras.layers.Dropout(rate=0.4)(conv2)
    
    conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)
    conv3 = keras.layers.Dropout(rate=0.5)(conv3)
    
    gap = keras.layers.GlobalAveragePooling1D()(conv3)
    
    output_layer = keras.layers.Dense(classes, activation='softmax')(gap)
    
    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(), 
        metrics=['accuracy'])
    
    return model

In [ ]:
# cnn model in pytorch
X=np.array(train_sc.iloc[:,2:]).reshape(3125, 600, -1)
X.shape
test_x=np.array(test_sc.iloc[:,2:]).reshape(782, 600, -1)
test_x.shape
y = train_labels['label'].values
# tf.keras.utils.to_categorical : Converts a class vector (integers) to binary class matrix.
y = tf.keras.utils.to_categorical(train_labels['label']) 
y.shape



In [ ]:
import torch.nn as nn

def cnn_model(input_shape, classes):
    seed(2021)
    시드 고정
    
    input_layer = 
    # 합성층 1
    conv1 = Conv1D(in_channels = 18, out_channels = 10 , kernel_size = 9)
    conv1 = nn.BatchNorm1d
    conv1 = nn.relu()
    conv1 = nn.Dropout(0.3)
    
    conv2 = Conv1D(in_channels = 10, out_channels = 5 , kernel_size = 6)
    conv2 = nn.BatchNorm1d
    conv2 = nn.relu()
    conv2 = nn.Dropout(0.4)
    
    conv3 = Conv1D(in_channels = 5, out_channels = 3 , kernel_size = 3)
    conv3 = nn.BatchNorm1d
    conv3 = nn.relu()
    conv3 = nn.Dropout(0.5)
# cnn model -> pytorch 구현
def cnn_model(input_shape, classes):
    seed(2021)
    tf.random.set_seed(2021)
    
    input_layer = keras.layers.Input(input_shape)
    conv1 = keras.layers.Conv1D(filters=128, kernel_size=9, padding='same')(input_layer) # feature map=128
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activation='relu')(conv1)
    conv1 = keras.layers.Dropout(rate=0.3)(conv1)

    conv2 = keras.layers.Conv1D(filters=256, kernel_size=6, padding='same')(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)
    conv2 = keras.layers.Dropout(rate=0.4)(conv2)
    
    conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)
    conv3 = keras.layers.Dropout(rate=0.5)(conv3)
    
    gap = keras.layers.GlobalAveragePooling1D()(conv3)
    
    output_layer = keras.layers.Dense(classes, activation='softmax')(gap)
    
    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(), 
        metrics=['accuracy'])
    
    return model

GAP의 목적은 feature를 1차원 벡터로 만들기 위함
GAP는 CNN + FC(Fully Connected) Layer에서 classifier인 FC Layer를 없애기 위한 방법으로 도입
어떤 사이즈의 입력이 들어와도 상관없고 학습, 과적합 유리
GAP 연산 결과는 1차원 벡터
경우에 따라서 FC layer와 같이 사용 되기도 합니다. FC layer에 전달하기 전에 GAP를 이용하여 차원을 줄여서 벡터로 만든 다음에 FC layer로 전달 하면 FC Layer에서 쉽게 사이즈를 맞출 수 있기 때문입니다.

10-fold StratifiedKFold (분류문제에서 label의 분포가 각 class별로 불균형을 이룰 때 유용)

In [43]:
skf = StratifiedKFold(n_splits = 10, random_state = 2021, shuffle = True)
reLR = ReduceLROnPlateau(patience = 4,verbose = 1,factor = 0.5) 
es =EarlyStopping(monitor='val_loss', patience=8, mode='min')

accuracy = []
losss=[]
models=[]

for i, (train, validation) in enumerate(skf.split(X, y.argmax(1))) :
    mc = ModelCheckpoint(f'./model_kf/cv_study{i + 1}.h5',save_best_only=True, verbose=0, monitor = 'val_loss', mode = 'min', save_weights_only=True)
    print("-" * 20 +"Fold_"+str(i+1)+ "-" * 20)
    model = cnn_model((600,18),61)
    history = model.fit(X[train], y[train], epochs = 100, validation_data= (X[validation], y[validation]), 
                        verbose=1,batch_size=64,callbacks=[es,mc,reLR])
    model.load_weights(f'./model_kf/cv_study{i + 1}.h5')
    
    k_accuracy = '%.4f' % (model.evaluate(X[validation], y[validation])[1])
    k_loss = '%.4f' % (model.evaluate(X[validation], y[validation])[0])
    
    accuracy.append(k_accuracy)
    losss.append(k_loss)
    models.append(model)

print('\nK-fold cross validation Auc: {}'.format(accuracy))
print('\nK-fold cross validation loss: {}'.format(losss))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\leesa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-43-4c219eb9d926>", line 1, in <module>
    skf = StratifiedKFold(n_splits = 10, random_state = 2021, shuffle = True)
NameError: name 'StratifiedKFold' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\leesa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\leesa\anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import 

NameError: name 'StratifiedKFold' is not defined

성능 확인 및 제출

In [44]:
print(sum([float(i) for i in accuracy])/10)
print()
print(sum([float(i) for i in losss])/10)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\leesa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-44-709c0f85d52a>", line 1, in <module>
    print(sum([float(i) for i in accuracy])/10)
NameError: name 'accuracy' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\leesa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\leesa\anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\leesa\anaconda3\li

NameError: name 'accuracy' is not defined

In [ ]:
test_X=np.array(test_sc.iloc[:,2:]).reshape(782, 600, -1)
test_X.shape

In [ ]:
preds = []
for model in models:
    pred = model.predict(test_X)
    preds.append(pred)
pred = np.mean(preds, axis=0)
pred

In [ ]:
submission=pd.read_csv('./data/sample_submission.csv')
submission.iloc[:,1:]=pred
submission
submission.to_csv('sub_kfold_stratified_10_adam_fft_0.5.csv',index=False)